In [1]:
# https://lightgbm.readthedocs.io/en/v3.3.2/Python-Intro.html
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
# https://coiled.io/blog/dask-dataframe-merge-join/


In [57]:
%pip install lightgbm --install-option=--gpu
import numpy as np
import lightgbm as lgb
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.distributed import LocalCluster, Client


/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# https://docs.dask.org/en/latest/deploying-python.html
# https://coiled.io/blog/dask-dataframe-merge-join/  << Ctrl + F "Run Massive Joins on Dask Cluster"
# Example: 
# https://github.com/microsoft/LightGBM/blob/fdc582ea6ba13faf15ee6707c7c7542790c8821d/examples/python-guide/dask/prediction.py
# https://examples.dask.org/machine-learning.html
# obj: create Dask cluster

# cluster = LocalCluster(n_workers=2, threads_per_worker=2, dashboard_address=':8383')
# client = Client(cluster)
# client.restart()
# print(client)
client.shutdown()

# dashboard: for task, cpu, worker load, must have bokeh installed
# https://docs.dask.org/en/stable/dashboard.html < diagnostics
# https://youtu.be/N_GqzcuGLCY

/home/ubuntu/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8383 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45145 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:36339' processes=2 threads=4, memory=59.86 GiB>


In [20]:
%%time

# types of joins: https://www.geeksforgeeks.org/how-to-merge-two-csv-files-by-specific-column-using-pandas-in-python/
# how to merge df w/ dask: https://coiled.io/blog/dask-dataframe-merge-join/
# obj: full outer join label csv to feature csv



# ProgressBar().register()
training_features = dd.read_csv("./originalDataset/train_data.csv", blocksize=25e6)
training_label = dd.read_csv("./originalDataset/train_labels.csv", blocksize=25e6)

training_merged = training_features.merge(
    training_label, 
    how="outer", 
    on=["customer_ID"]
)
# training_merged.compute()
# training_merged.head(n=5)




distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

CPU times: user 7min 9s, sys: 2min 16s, total: 9min 26s
Wall time: 24min 20s


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,00ee24c2957b6cb345dc5176469579631188e298174898...,2017-03-08,0.674533,0.007463,0.381584,0.016094,0.004984,NaN,0.249030,0.551853,...,NaN,NaN,0.001170,0.001337,0.004494,NaN,0.003368,0.003466,0.004135,1
1,00ee24c2957b6cb345dc5176469579631188e298174898...,2017-04-07,0.674562,0.000663,0.372331,0.013610,0.000699,NaN,0.221530,0.581035,...,NaN,NaN,0.002111,0.001737,0.006009,NaN,0.001264,0.003623,0.007882,1
2,00ee24c2957b6cb345dc5176469579631188e298174898...,2017-05-08,0.680785,0.001757,0.365993,0.017561,0.001139,NaN,0.199713,0.604646,...,NaN,NaN,0.005169,0.006371,0.000670,NaN,0.003822,0.003353,0.002635,1
3,00ee24c2957b6cb345dc5176469579631188e298174898...,2017-06-07,0.692038,0.032824,0.364222,0.011793,0.002008,NaN,0.175190,0.628946,...,NaN,NaN,0.005760,0.009068,0.000025,NaN,0.005024,0.004344,0.008673,1
4,00ee24c2957b6cb345dc5176469579631188e298174898...,2017-07-08,0.655387,0.036537,0.360272,0.012637,0.000292,NaN,0.002800,0.648046,...,NaN,NaN,0.000384,0.004604,0.009883,NaN,0.000203,0.004457,0.008245,1


distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)


In [ ]:
%%time

training_merged.to_csv("./originalDataset/training_merged.csv")

In [58]:
%%time
# feature engineering
categorical_feature_array=[
        "B_30", "B_38", "D_114", 
        "D_116", "D_117", "D_120",
        "D_126", "D_63", "D_64",
        "D_66", "D_68"
]

# def feature_engineering(dataframe):
#         for c in categorical_feature_array:
#                 dataframe[c] = dataframe[c].astype('category')
#         return dataframe

# training_merged = feature_engineering(training_merged)
training_merged = dd.read_csv("./originalDataset/training_merged.csv/*.part")

CPU times: user 307 ms, sys: 114 ms, total: 421 ms
Wall time: 324 ms


In [69]:
%%time
# features = training_merged[training_merged.columns[training_merged.columns!="target"]]
# label = training_merged["target"]

# must create training set, validation set, & test set using training_merged to train, validate, & test model



training_set, validation_set, test_set = training_merged.random_split([0.8, 0.1, 0.1], random_state=123)

train_x = training_set.drop(columns=["target","customer_ID"])
train_y = training_set["target"]

val_x = validation_set.drop(columns=["target","customer_ID"])
val_y = validation_set["target"]

test_x = test_set.drop(columns=["target","customer_ID"])
test_y = test_set["target"]


CPU times: user 189 ms, sys: 33.9 ms, total: 223 ms
Wall time: 186 ms


In [64]:
%%time

# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html
# https://lightgbm.readthedocs.io/en/latest/Parallel-Learning-Guide.html#dask
# https://www.analyticsvidhya.com/blog/2021/08/complete-guide-on-how-to-use-lightgbm-in-python/
# model development



# https://lightgbm.readthedocs.io/en/latest/Parameters.html
# https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
# https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html#for-better-accuracy



params = {
    # change accuracy, use dart bosoting type?
    "num_leaves" : 31,
    "max_bin" : 63,
    "learning_rate" : 0.1,
    "num_iterations" : 100,
    "boosting_type" : "goss",

    # change overfitting
    "max_depth" : -1,
    "min_split_gain" : 0,
    "min_child_samples" : 20,
    "reg_alpha" : 0,
    "reg_lambda" : 0,
    "early_stopping" : 5,
    "feature_pre_filter" : True,

    # 20x weight on negative
    "scale_pos_weight" : 0.05,

    # gpu
    "device_type" : "gpu",

    # misc
    "verbose" : -1
}

dask_model = lgb.DaskLGBMClassifier(
    objective="binary",
    n_jobs=-1,
    random_state=42,
    **params
)



CPU times: user 76 µs, sys: 0 ns, total: 76 µs
Wall time: 83.9 µs


In [66]:
%%time

dask_model.fit(
    train_x,
    train_y,
    eval_set=[(val_x, val_y),(train_x, train_y)],
    eval_metric="logloss",
    categorical_feature=categorical_feature_array,
    tree_learner="voting"
)

print(f"Training accuracy: {dask_model.score(train_x, train_y)}")
print(f"Validation accuracy: {dask_model.score(val_x, val_y)}")
print(f"Test accuracy: {dask_model.score(test_x, test_y)}")

KeyboardInterrupt: 

In [ ]:
# tuning https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

In [ ]:
%%time
# https://www.analyticsvidhya.com/blog/2021/08/complete-guide-on-how-to-use-lightgbm-in-python/
# Additional
from sklearn.metrics import classification_report
lgb.plot_importance(dask_model)
lgb.plot_metric(dask_model)
print(classification_report(test_y,dask_model.predict(test_x)))

In [ ]:
%%time
# make predictions on test_data features and reference sample_submission for format
test_features_with_ID = dd.read_csv("./originalDataset/test_data.csv", blocksize=25e6)

# changes features to match what was used to train model
# test_features_with_ID = feature_engineering(test_features_with_ID)

test_features = test_features_with_ID.drop(columns=["customer_ID"])
pred_y = dask_model.predict(test_features)
final_submit_df = test_features_with_ID["customer_ID"]
final_submit_df["prediction"] = pred_y
final_submit_df.to_csv("final_submit.csv", single_file=True)

# in cli
# kaggle competitions submit -c [COMPETITION] -f [FILE] -m [MESSAGE]
!kaggle competitions submit -c amex-default-prediction -f final_submit.csv -m "first submit"

In [ ]:
# pickle model when done?